# FAKE NEWS LSTM

## Imports

In [1]:
from argparse import Namespace
import os
import json

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

In [2]:
from collections import Counter
import re
import string

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords # remove stopword
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from nltk import word_tokenize

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


## Data Vectorization classes

### Vocabulary

In [4]:
class Vocabulary(object):
    """Class to process text and extract vocabulary for mapping"""

    def __init__(self, token_to_idx=None):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
        """    
        # _token_to_idx for char_vocab:
        #   {'<MASK>': 0, '<UNK>': 1, '<BEGIN>': 2, '<END>': 3, 'T': 4, 'o': 5, 't': 6, 'a': 7, ....., 'Á': 79}
        # _idx_to_token: 
        #   {0:'<MASK>'0, 1:'<UNK>', 2:'<BEGIN>',   3:'<END>',   4:'T',  5:'o',  6:'t',  7:'a',....., 79:'Á'}
            
        # _token_to_idx nationality_vocab: {'Arabic': 0, 'Chinese': 1, ..., 'Vietnamese': 17}
        # _idx_to_token:                   {0:'Arabic',  1:'Chinese', ...,  17:'Vietnamese'}

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx         
                                                  
        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
        
    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
            
    def lookup_token(self, token):
        """Retrieve the index associated with the token 
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        """
        return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index
        
        Args: 
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

In [5]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token  # for paddding, e.g., 'McMahan' -> [2, 5, 6, 5, 7, 8, 7, 9, 3, 0, 0, 0, ..., 0]
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)           # mask_index is 0
        self.unk_index = self.add_token(self._unk_token)             # unk_index is 1
        self.begin_seq_index = self.add_token(self._begin_seq_token) # begin_seq_index is 2
        self.end_seq_index = self.add_token(self._end_seq_token)     # end_seq_index is 3

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

### Vectorizer

In [6]:
class NewsVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""   
    def __init__(self, news_vocab, category_vocab):
        self.news_vocab = news_vocab
        self.category_vocab = category_vocab

    def vectorize(self, text, vector_length=-1):
        """
        Args:
            text (str): the string of words separated by a space
            vector_length (int): an argument for forcing the length of index vector
        Returns:
            the vetorized text (numpy.array)
        """        
        indices = [self.news_vocab.begin_seq_index]
        indices.extend(self.news_vocab.lookup_token(token) 
                       for token in text.split(" "))
        indices.append(self.news_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices)

        out_vector = np.zeros(vector_length, dtype=np.int64)   # SurnameDataset._max_seq_length is 19 in current dataset.      
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.news_vocab.mask_index
        
        """    
        mask_index is 0
        unk_index is 1
        begin_seq_index is 2
        end_seq_index is 3
        
        When surname is "McMahan", M=5,c=6, a=7, h=8, and n=9 
        
        out_vector = [2, 5, 6, 5, 7, 8, 7, 9, 3, 0, 0, 0, ..., 0]
        len(indices) = 9      
        """ 
        #return out_vector, len(indices)
        return out_vector

    @classmethod
    def from_dataframe(cls, news_df, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        Args:
            news_df (pandas.DataFrame): the surnames dataset
        Returns:
            an instance of the NewsVectorizer
        """
        category_vocab = Vocabulary()
        for category in sorted(set(news_df.category)):
            category_vocab.add_token(category)

        word_counts = Counter()
        for text in news_df.text:
            for token in text.split(" "):
                if token not in string.punctuation:
                    word_counts[token] += 1
                    
        news_vocab = SequenceVocabulary()
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                news_vocab.add_token(word)
        
        return cls(news_vocab, category_vocab)

### Dataset

In [7]:
def text_preprocessing(text):
    text = text.lower() # case folding
    text = re.sub('&\w*\;\w*', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub(r"[^a-z]+", r" ", text) # Regulation, remove special charecters
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    lemmatizer=WordNetLemmatizer()
    # remove stopwords and lemmatization
    result = [lemmatizer.lemmatize(i) for i in tokens if not i in stop_words]
    result = result[:500]
    # bigram
    bigram_result = []
    bigram_list = ngrams(result, 2)
    for word_set in bigram_list:
        for word in word_set:
            bigram_result.append(word)
    return bigram_result

test_str_1 = "This is sentence to test the effect of preprocessing... Yeah~\nCool!fac  feae ge fe ga 🪣🛀🎀 ☏☢︎⏥␘⍎ (>╹ω╹<)喵"
test_str_2 = "Glaxo Settles Paxil 'Suicide Pill' Suit.  NEW YORK (Reuters) - GlaxoSmithKline Plc &lt;A HREF=http://www.investor.reuters.com/FullQuote.aspx?ticker=GSK.L target=/stocks/quickinfo/fullquote""&gt;GSK.L&lt;/A&gt; has agreed  to release all clinical studies of its drugs to settle a  lawsuit that accused it of withholding negative information  about the antidepressant Paxil, the New York Attorney General's  office said on Thursday."
list = text_preprocessing(test_str_2)
' '.join(list)

'glaxo settle settle paxil paxil suicide suicide pill pill suit suit new new york york reuters reuters glaxosmithkline glaxosmithkline plc plc href href target target stock stock quickinfo quickinfo fullquote fullquote l l agreed agreed release release clinical clinical study study drug drug settle settle lawsuit lawsuit accused accused withholding withholding negative negative information information antidepressant antidepressant paxil paxil new new york york attorney attorney general general office office said said thursday'

In [8]:
class NewsDataset(Dataset):
    def __init__(self, news_df, vectorizer):
        """
        Args:
            news_df (pandas.DataFrame): the dataset
            vectorizer (SurnameVectorizer): vectorizer instatiated from dataset
        """
        self.news_df = news_df 
        self._vectorizer = vectorizer
        
        # +1 if only using begin_seq, +2 if using both begin and end seq tokens
        measure_len = lambda context: len(context.split(" "))
        self._max_seq_length = max(map(measure_len, news_df.text)) + 2

        self.train_df = self.news_df[self.news_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.news_df[self.news_df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.news_df[self.news_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size), 
                             'val': (self.val_df, self.validation_size), 
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')
        
        # Class weights
        class_counts = self.train_df.category.value_counts().to_dict()  
        def sort_key(item):
            return self._vectorizer.category_vocab.lookup_token(item[0]) 
        sorted_counts = sorted(class_counts.items(), key=sort_key)          
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32) 

        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, news_csv):
        """Load dataset and make a new vectorizer from scratch
        
        Args:
            news_csv (str): location of the dataset
        Returns:
            an instance of SurnameDataset
        """
        news_df = pd.read_csv(news_csv)
        for index, text in enumerate(news_df.text):
            processed_list = text_preprocessing(text)
            news_df.text[index] = ' '.join(processed_list)
        train_news_df = news_df[news_df.split=='train']
        return cls(news_df, NewsVectorizer.from_dataframe(train_news_df))
        
    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's:
                features (x_data)
                label (y_target)
                feature length (x_length)
        """
        row = self._target_df.iloc[index]
        
        news_vector = \
            self._vectorizer.vectorize(row.text, self._max_seq_length)
        
        category_index = \
            self._vectorizer.category_vocab.lookup_token(row.category)

        return {'x_data': news_vector, 
                'y_target': category_index} 
    
    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size   

def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"): 
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

## Model: NewsClassifier

In [9]:
def column_gather(y_out, x_lengths):
    '''Get a specific vector from each batch datapoint in `y_out`.

    More precisely, iterate over batch row indices, get the vector that's at
    the position indicated by the corresponding value in `x_lengths` at the row
    index.

    Args:
        y_out (torch.FloatTensor, torch.cuda.FloatTensor)
            shape: (batch, sequence, feature)
        x_lengths (torch.LongTensor, torch.cuda.LongTensor)
            shape: (batch,)

    Returns:
        y_out (torch.FloatTensor, torch.cuda.FloatTensor)
            shape: (batch, feature)
    '''
    x_lengths = x_lengths.long().detach().cpu().numpy() - 1   # deduct 1 since the index starts from 0
                                                              # e.g., [9, 6, 11, 9, 7, ...., 12]
    out = []
    for batch_index, column_index in enumerate(x_lengths): # out gets the last hidden vector of each input: (batch, hidden_size)
        out.append(y_out[batch_index, column_index]) # e.g., y_out[0, 9], y_out[1, 6]

    return torch.stack(out)  # (batch, hidden_size*num_directions); E.g., (64, 64*num_direction)

def column_summation(y_out, x_lengths, mode="mean"):
    '''Get a max or mean vector from each batch datapoint in `y_out`.

    More precisely, iterate over batch row indices, get the max or mean vector of all the vectors by 
    the position indicated by the corresponding value in `x_lengths` at the row index.

    Args:
        y_out (torch.FloatTensor, torch.cuda.FloatTensor)
            shape: (batch, sequence, feature)
        x_lengths (torch.LongTensor, torch.cuda.LongTensor)
            shape: (batch,)
        mode: "mean" for mean vector; "max" for max vector

    Returns:
        y_out (torch.FloatTensor, torch.cuda.FloatTensor)
            shape: (batch, feature)
    '''
    x_lengths = x_lengths.long().detach().cpu().numpy() - 1

    out = []
    for batch_index, column_index in enumerate(x_lengths):
        if mode == "mean":
            # replace "pass" with your code to get the mean vector of current batch item from y_out[], and append it to out list.
            pass
        else:
            # replace "pass" with your code to get the max vector of current batch item from y_out[], , and append it to out list.
            pass

    return torch.stack(out)

In [10]:
'''class NewsClassifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings, num_classes,
                 rnn_hidden_size, bidirectional=False, batch_first=True, padding_idx=0):
        """
        Args:
            embedding_size (int): The size of the character embeddings 
            num_embeddings (int): The number of characters to embed 9590
            num_classes (int): The size of the prediction vector 
            bidirectional (bool): Informs whether bidrectional RNN is used
            rnn_hidden_size (int): The size of the RNN's hidden state
            batch_first (bool): Informs whether the input tensors will 
                have batch or the sequence on the 0th dimension
            padding_idx (int): The index for the tensor padding; 
                see torch.nn.Embedding
        """
        super(NewsClassifier, self).__init__()

        if bidirectional == False:
             self.num_directions = 1
        else:
             self.num_directions = 2
        
        self.emb = nn.Embedding(num_embeddings=num_embeddings,    # E.g., (80, 100)
                                embedding_dim=embedding_size,
                                padding_idx=padding_idx)          # mask_index (padding index) is 0

        #self.rnn = nn.RNN(input_size=embedding_size,              # E.g., 100
        #self.rnn = nn.GRU(input_size=embedding_size,
        self.rnn = nn.LSTM(input_size=embedding_size,
                             hidden_size=rnn_hidden_size,         # E.g., 64
                             batch_first=batch_first, 
                             num_layers = 2,
                             dropout = 0.0, 
                             bidirectional=bidirectional)

        
        self.fc1 = nn.Linear(in_features=rnn_hidden_size*self.num_directions, out_features=rnn_hidden_size*self.num_directions)
        self.fc2 = nn.Linear(in_features=rnn_hidden_size*self.num_directions,out_features=num_classes)                            
        # for batch norm
        self.bn1 = nn.BatchNorm1d(rnn_hidden_size*self.num_directions) 


    def forward(self, x_in, x_lengths=None, apply_softmax=False):
        """The forward pass of the classifier
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, input_dim, i.e. seq_size)
            x_lengths (torch.Tensor): the lengths of each sequence in the batch.
                They are used to find the final vector of each sequence: (batch,)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, output_dim)
        """
        x_embedded = self.emb(x_in)    
        
        y_out, _ = self.rnn(x_embedded)  
      
        if x_lengths is not None:       
            y_out = column_gather(y_out, x_lengths) 
        else:
            y_out = y_out[:, -1, :]      
            
        # with batch norm and dropout
        y_out = F.relu(self.bn1(self.fc1(F.dropout(y_out, 0.2, training=self.training))))  # y_out: (64, 64*num_direction)

        
        # with dropout
        y_out = self.fc2(F.dropout(y_out, 0.2, training=self.training))   # y_out: (batch, num_classes) ; (64, 18)

        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)

        return y_out'''

'class NewsClassifier(nn.Module):\n    def __init__(self, embedding_size, num_embeddings, num_classes,\n                 rnn_hidden_size, bidirectional=False, batch_first=True, padding_idx=0):\n        """\n        Args:\n            embedding_size (int): The size of the character embeddings \n            num_embeddings (int): The number of characters to embed 9590\n            num_classes (int): The size of the prediction vector \n            bidirectional (bool): Informs whether bidrectional RNN is used\n            rnn_hidden_size (int): The size of the RNN\'s hidden state\n            batch_first (bool): Informs whether the input tensors will \n                have batch or the sequence on the 0th dimension\n            padding_idx (int): The index for the tensor padding; \n                see torch.nn.Embedding\n        """\n        super(NewsClassifier, self).__init__()\n\n        if bidirectional == False:\n             self.num_directions = 1\n        else:\n             self.n

In [11]:

class NewsClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, num_classes, rnn_hidden_size, num_layers, bidirectional, pretrained_embeddings=None):
        super(NewsClassifier, self).__init__()

        if pretrained_embeddings is None:
            self.embedding = nn.Embedding(num_embeddings, embedding_dim)
            self.embedding.weight.requires_grad = True
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=False)

        self.lstm = nn.LSTM(embedding_dim, rnn_hidden_size, num_layers=num_layers, bidirectional=bidirectional, batch_first=True)
        self.fc = nn.Linear(rnn_hidden_size * 2 if bidirectional else rnn_hidden_size, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, cell) = self.lstm(embedded)
        if self.lstm.bidirectional:
            hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        else:
            hidden = hidden[-1, :, :]
        out = self.fc(hidden)
        return out

## Settings

### general utilities

In [12]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

In [13]:
def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
        
def load_glove_from_file(glove_filepath):
    """
    Load the GloVe embeddings 
    
    Args:
        glove_filepath (str): path to the glove embeddings file 
    Returns:
        word_to_index (dict), embeddings (numpy.ndarary)
    """

    word_to_index = {}
    embeddings = []
    with open(glove_filepath, "r", encoding='utf8') as fp:
        for index, line in enumerate(fp):
            line = line.split(" ") # each line: word num1 num2 ...
            word_to_index[line[0]] = index # word = line[0] 
            embedding_i = np.array([float(val) for val in line[1:]])
            embeddings.append(embedding_i)
    return word_to_index, np.stack(embeddings)

def make_embedding_matrix(glove_filepath, words):
    """
    Create embedding matrix for a specific set of words.
    
    Args:
        glove_filepath (str): file path to the glove embeddigns
        words (list): list of words in the dataset
    """
    word_to_idx, glove_embeddings = load_glove_from_file(glove_filepath)
    embedding_size = glove_embeddings.shape[1]
    
    final_embeddings = np.zeros((len(words), embedding_size))

    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = glove_embeddings[word_to_idx[word]]
        else:
            embedding_i = torch.ones(1, embedding_size)
            torch.nn.init.xavier_uniform_(embedding_i)
            final_embeddings[i, :] = embedding_i

    return final_embeddings

In [14]:
args = Namespace(
    # Data and path information
    news_csv="../data/processed/fake_or_real_news_with_splits.csv",
    model_state_file="model_LSTM_Fake_News.pth",
    save_dir="../model_storage/Fake_News",
    # Model hyper parameter
    glove_filepath='../data/glove/glove.6B.100d.txt', 
    use_glove=True,
    embedding_size=100,
    rnn_hidden_size=128,
    bidirectional=True,
    num_layers=1,
    # Training hyper parameter
    num_epochs=30,
    learning_rate=0.001,
    batch_size=32,
    seed=1337,
    early_stopping_criteria=10,
    # Runtime hyper parameter
    cuda=True,
    catch_keyboard_interrupt=True,
    expand_filepaths_to_save_dir=True,
)


    
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
    
print("Using CUDA: {}".format(args.cuda))


if args.expand_filepaths_to_save_dir:
    args.model_state_file = os.path.join(args.save_dir, args.model_state_file)
    
# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# handle dirs
handle_dirs(args.save_dir)

Using CUDA: True


### Initializations

In [15]:
# create dataset and vectorizer
dataset = NewsDataset.load_dataset_and_make_vectorizer(args.news_csv)

vectorizer = dataset.get_vectorizer()

# Use GloVe or randomly initialized embeddings
if args.use_glove:
    words = vectorizer.news_vocab._token_to_idx.keys()  # 3409 unique words
    embeddings = make_embedding_matrix(glove_filepath=args.glove_filepath,     # embeddings: (3409, 100)
                                       words=words)
    print("Using pre-trained embeddings")
else:
    print("Not using pre-trained embeddings")
    embeddings = None

C:\Users\Croya\AppData\Local\Temp\ipykernel_22852\239385863.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df.text[index] = ' '.join(processed_list)


Using pre-trained embeddings


In [16]:
classifier = NewsClassifier(   num_embeddings=len(vectorizer.news_vocab),
                               embedding_dim=args.embedding_size, 
                               num_classes=len(vectorizer.category_vocab),
                               rnn_hidden_size=args.rnn_hidden_size,
                               num_layers=args.num_layers,
                               bidirectional=args.bidirectional,
                               #padding_idx=vectorizer.news_vocab.mask_index
                               )

In [17]:
print(classifier)

NewsClassifier(
  (embedding): Embedding(9590, 100)
  (lstm): LSTM(100, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)


In [18]:
dataset._max_seq_length # max sequence length

1000

In [19]:
vectorizer.news_vocab._token_to_idx

{'<MASK>': 0,
 '<UNK>': 1,
 '<BEGIN>': 2,
 '<END>': 3,
 'nothing': 4,
 'good': 5,
 'come': 6,
 'election': 7,
 'posted': 8,
 'november': 9,
 'charles': 10,
 'hugh': 11,
 'smith': 12,
 'overwhelming': 13,
 'consensus': 14,
 'across': 15,
 'political': 16,
 'spectrum': 17,
 'thing': 18,
 'go': 19,
 'like': 20,
 'country': 21,
 'deeply': 22,
 'divided': 23,
 'every': 24,
 'institution': 25,
 'two': 26,
 'party': 27,
 'mainstream': 28,
 'medium': 29,
 'department': 30,
 'justice': 31,
 'cover': 32,
 'ups': 33,
 'collusion': 34,
 'worse': 35,
 'whomever': 36,
 'win': 37,
 'enter': 38,
 'presidency': 39,
 'without': 40,
 'mandate': 41,
 'nation': 42,
 'corruption': 43,
 'require': 44,
 'hitting': 45,
 'bottom': 46,
 'alcoholic': 47,
 'drug': 48,
 'incapable': 49,
 'making': 50,
 'truly': 51,
 'positive': 52,
 'change': 53,
 'hit': 54,
 'absolute': 55,
 'tolerance': 56,
 'corrupt': 57,
 'system': 58,
 'poisoning': 59,
 'one': 60,
 'injection': 61,
 'cash': 62,
 'pay': 63,
 'play': 64,
 'time'

In [20]:
vectorizer.category_vocab._token_to_idx

{'FAKE': 0, 'REAL': 1}

## Training Routine

### Helper functions

In [21]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}


def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better
    
    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]
         
        # If loss worsened
        #if loss_t >= loss_tm1:
        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state


def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

### Training loop

In [ ]:
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
    
loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.1, patience=10)

train_state = make_train_state(args)

epoch_bar = tqdm(desc='training routine', total=args.num_epochs, position=0)

dataset.set_split('train')
train_bar = tqdm(desc='split=train', total=dataset.get_num_batches(args.batch_size), position=1, leave=True)

dataset.set_split('val')
val_bar = tqdm(desc='split=val', total=dataset.get_num_batches(args.batch_size), position=1, leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------    
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = classifier(batch_dict['x_data'])  
            
            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
    
            running_loss += (loss.item() - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on

        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            # compute the output
            y_pred = classifier(batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            running_loss += (loss.item() - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            val_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier, 
                                         train_state=train_state)


        scheduler.step(train_state['val_loss'][-1])

        train_bar.n = 1
        val_bar.n = 1
        epoch_bar.update()

        if train_state['stop_early']:
            break
            
except KeyboardInterrupt:
    print("Exiting loop")

training routine:   0%|          | 0/30 [00:00<?, ?it/s]

split=train:   0%|          | 0/138 [00:00<?, ?it/s]

split=val:   0%|          | 0/29 [00:00<?, ?it/s]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

acc = train_state['train_acc']
val_acc = train_state['val_acc']
loss = train_state['train_loss']
val_loss = train_state['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename'])) # load the best model

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

y_pred_list = []         # store predicted values for confusion matrix
y_category_list = []  # ground truth value

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(batch_dict['x_data'])
    
    # store predicted values and ground truth values for calculating confusion matrix
    y_pred_list.extend(y_pred.max(dim=1)[1].cpu().numpy())
    y_category_list.extend(batch_dict['y_target'].cpu().numpy())
    
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc


In [ ]:
print("Test loss: {};".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
category_classes = []
for i in range(len(dataset._vectorizer.category_vocab)):
    category_classes.append(dataset._vectorizer.category_vocab.lookup_index(i))
print(category_classes)

In [ ]:
import pandas as pd

cm = confusion_matrix(y_category_list, y_pred_list)
cm_df = pd.DataFrame(cm.T, index=category_classes, columns=category_classes)
cm_df.index.name = 'Predicted'
cm_df.columns.name = 'True'
print(cm_df)

In [ ]:
print(classification_report(y_category_list, y_pred_list))

### Inference

In [ ]:
# Preprocess the reviews
def preprocess_text(text):
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

In [ ]:
def predict_category(title, classifier, vectorizer, max_length):
    """Predict a News category for a new title
    
    Args:
        title (str): a raw title string
        classifier (NewsClassifier): an instance of the trained classifier
        vectorizer (NewsVectorizer): the corresponding vectorizer
        max_length (int): the max sequence length
            Note: CNNs are sensitive to the input data tensor size. 
                  This ensures to keep it the same size as the training data
    """
    title = preprocess_text(title)
    vectorized_title = \
        torch.tensor(vectorizer.vectorize(title, vector_length=max_length))
    result = classifier(vectorized_title.unsqueeze(0))
    probability_values, indices = result.max(dim=1)
    predicted_category = vectorizer.category_vocab.lookup_index(indices.item())

    return {'category': predicted_category, 
            'probability': probability_values.item()}

In [ ]:
def get_samples():
    samples = {}
    for cat in dataset.val_df.category.unique():
        samples[cat] = dataset.val_df.text[dataset.val_df.category==cat].tolist()[:5]
    return samples

val_samples = get_samples() # first 5 titles of each category from validation dataset

In [ ]:
#title = input("Enter a news title to classify: ")
classifier = classifier.to("cpu")

for truth, sample_group in val_samples.items():
    print(f"True Category: {truth}")
    print("="*30)
    for sample in sample_group:
        prediction = predict_category(sample, classifier, 
                                      vectorizer, dataset._max_seq_length)
        print("Prediction: {} (p={:0.2f})".format(prediction['category'],
                                                  prediction['probability']))
        print("\t + Sample: {}".format(sample))
    print("-"*30 + "\n")